<br>

# Introdução

In [ ]:
#!pip3 install traquitanas --upgrade
#!pip3 install -r requirements.txt
#!/usr/local/bin/python -m pip install --upgrade pip

In [ ]:
import os
import pprint
import shutil
import folium
import fnmatch
import pandas as pd
import geopandas as gpd

In [ ]:
from osgeo import ogr
from folium import plugins
from zipfile import ZipFile

In [ ]:
from traquitanas.geo import layers as tt

In [ ]:
from paths import *

<br>

Inicialmente extraímos todos os arquivos de dentro do zip "Shape.zip".

In [ ]:
zipfile_path = input_path / 'Shapes.zip'

with ZipFile(zipfile_path, 'r') as zip_obj:    
    zip_obj.extractall(temp_path)

<br>

Uma vez com os arquivos soltos, foi possível observar grande desorganização dos arquivos. Não há qualquer padrão de nomenclatura. Alguns *shapefiles* existem para algumas nascentes, enquanto para outra snão existe.<br>
Como o objetivo principal do trabalho é identificar a localização das 56 nascentes que são mencionadas no TAC, optou-se por concentrar os esforços nestas feições.

In [ ]:
list_shps = []

for root, dirs, files in os.walk(temp_path):
    if os.path.basename(root).startswith('Nascente'):
        #print(f'\n{root}')
        for file in fnmatch.filter(files, '*.shp'):
            #print(f'> {file}')
            list_shps.append(file)
            pass

list_shps = list(set(list_shps))
list_shps.sort()
list_shps

In [ ]:
list_shps = list(set([x.name for x in list(temp_path.glob('Nascente*/**/*.shp'))]))
list_shps.sort()
pprint.pprint(list_shps)

a = list_shps[4]
print(a)

In [ ]:
list_files = list(set([x.name for x in list(temp_path.glob(f'Nascente*'))]))
list_files

In [ ]:

    
#     for root, dirs, files in os.walk(in_path):
#         if os.path.basename(root).startswith('Nascente'):
#             for file in fnmatch.filter(files, '*.shp'):
#                 if file in [shp]:
#                     # Read
#                     temp = os.path.join(root, file)
                    
                    
                    
                    
#                     gdf = gpd.read_file(temp)
#                     gdf.columns = [x.lower() for x in gdf.columns]

#                     # Delete Fields
#                     #for col in ['Id', 'in_path', 'nascente']:
#                     gdf.drop([
#                         'Id', 'in_path',
#                         'nascente', 'Nascente',
#                         'numero', 'x', 'y',
#                         'BUFF_DIST', 'buff_dist',
#                         'nascente_1'
#                         'nascente_1', 'nascente_2',
#                         'nascente_3', 'nascente_4',
#                         'nascente_5'],
#                         axis=1,
#                         errors='ignore',
#                         inplace=True,
#                     )
                    
#                     # Add Fields
#                     gdf['in_path'] = temp
#                     dirname = os.path.dirname(temp)
#                     dirname = os.path.basename(dirname)
#                     nascente_number = int(dirname.split(' ', maxsplit=1)[-1])
#                     gdf['nascente'] = nascente_number

#                     # Save Output
#                     gdf.to_file(temp)

#     # Lista Files
#     print(gdf.columns)
#     list_files = []                
#     for root, dirs, files in os.walk(in_path):
#         if os.path.basename(root).startswith('Nascente'):
#             for file in fnmatch.filter(files, '*.shp'):
#                 if file in [shp]:
#                     temp = os.path.join(root, file)
#                     list_files.append(temp)


<br>

## Junta shapefiles por Nome

Com a lista dos possíveis nomes de shapefiles, juntou-se todos os arquivos.

In [ ]:
def merge_shp_by_name(shp, in_path, out_path):
    """
    in_path: pasta root que tem os shapefiles
    out_path: pasta que salva
    
    """
    # Add Fields
    list_files = list(set([x for x in list(in_path.glob(f'Nascente*/**/{shp}'))]))
        
    # Combine all shapefiles
    gdf = gpd.GeoDataFrame(
        pd.concat(
            [gpd.read_file(i) for i in list_files],
            ignore_index=True
        ),
        crs=gpd.read_file(list_files[0]).crs
    )
    
    # Adjust Columns
    gdf.columns = [x.lower() for x in gdf.columns]

    # Delete Fields
    gdf.drop(
        [
            'Id',
            'in_path',
            'nascente',
            'Nascente',
            'numero',
            'x',
            'y',
            'BUFF_DIST',
            'buff_dist',
            'nascente_1'
            'nascente_1',
            'nascente_2',
            'nascente_3',
            'nascente_4',
            'nascente_5'
        ],
        axis=1,
        errors='ignore',
        inplace=True,
    )

    # Add Fields
    # gdf['in_path'] = temp
    # dirname = os.path.dirname(temp)
    # dirname = os.path.basename(dirname)
    # nascente_number = int(dirname.split(' ', maxsplit=1)[-1])
    # gdf['nascente'] = nascente_number

    # Combine all shapefiles
    gdf = gpd.GeoDataFrame(
        pd.concat(
            [gpd.read_file(i) for i in list_files],
            ignore_index=True
        ),
        crs=gpd.read_file(list_files[0]).crs
    )

    # Save Output
    geopackage = shp.replace('shp', 'gpkg')
    geopackage_file = out_path / f'{geopackage}'
    geopackage_file.unlink()
    gdf.to_file(geopackage_file, driver="GPKG")
    
    #display(gdf)
    return gdf

In [ ]:
for shp in list_shps:
    gdf = merge_shp_by_name(shp, temp_path, geo_path)

<br>

## Junta shapefiles Específicos

Após juntar todos os shapefiles em pastas, notei a existência de shapefiles que tem o mesmo tema, porém continham nomes distintos (por exemplo, "app.shp" e "App.shp").<br>
Esse problema na nomenclatura dos arquivos, impediu que os arquivos fossem unificados e, portanto, foi necessário escrever uma função que juntava os shapefiles, mantendo apenas o primeiro e deletando os restantes.

In [ ]:
def merge_shp_specific(list_files):
    """
    Une shapefiles específicos, sobrepondo o primeiro da lista...
    Deletando os demais...
    """
    # Concatena
    gdf = gpd.GeoDataFrame(
        pd.concat(
            [gpd.read_file(i) for i in list_files],
            ignore_index=True
        ),
        crs=gpd.read_file(list_files[0]).crs
    )

    # Deleta Others Files
    driver = ogr.GetDriverByName('ESRI Shapefile')
    for file in list_files[1:]:
        if os.path.exists(file):
            driver.DeleteDataSource(file)
            print(f'"{file}" deletado!!')
        else:
            print(f'"{file}" não existe!')

    # Save Output
    gdf.to_file(os.path.join(list_files[0]))
    return gdf.explore()

In [ ]:
merge_shp_specific(
    [
        os.path.join(shps_path, 'APP.shp'),
        os.path.join(shps_path, 'APP_nascente.shp'),
    ]
)

In [ ]:
merge_shp_specific(
    [
        os.path.join(shps_path, 'Area_conservacao.shp'),
        os.path.join(shps_path, 'area_Conservacao.shp'),
    ]
)

In [ ]:
merge_shp_specific(
    [
        os.path.join(shps_path, 'area_drenagem.shp'),
        os.path.join(shps_path, 'Area_Drenagem.shp'),
        os.path.join(shps_path, 'Area_drenagem.shp'),        
    ]
)

In [ ]:
merge_shp_specific(
    [
        os.path.join(shps_path, 'Recuperacao_curso_agua.shp'),
        os.path.join(shps_path, 'Recuperacao_curso_dagua.shp'),
        os.path.join(shps_path, 'Recuperacao_cursodagua.shp'),        
    ]
)

<br>

## Deleta Shapefiles

Após juntar o material que considerei útil para a etapa do meu trabalho, foi possível deletar o restante do material.

In [ ]:
def delete_shp(list_files):
    """
    Delete Lista de Shapefiles
    """
    # Deleta Others Files
    driver = ogr.GetDriverByName('ESRI Shapefile')
    for file in list_files[1:]:
        if os.path.exists(file):
            driver.DeleteDataSource(file)
            print(f'"{file}" deletado!!')
        else:
            print(f'"{file}" não existe!')


In [ ]:
list_files = [
    # Lixo
    os.path.join(shps_path, 'APP_total.shp'),

    # Interessa?
    os.path.join(shps_path, 'Geologia.shp'),
    #os.path.join(shps_path, 'Nascente.shp'),
    os.path.join(shps_path, 'Pontos_geofisica.shp'),
    os.path.join(shps_path, 'Recomposicao.shp'),
    os.path.join(shps_path, 'Recuperacao_nascente.shp'),
    os.path.join(shps_path, 'Uso_solo_APP.shp'),
    os.path.join(shps_path, 'Uso_solo_app_curso_dagua.shp'),
    os.path.join(shps_path, 'Uso_solo_app_nasc.shp'),
    os.path.join(shps_path, 'Uso_solo_area_drenagem.shp'),
    os.path.join(shps_path, 'Uso_solo_curso_agua.shp'),
    os.path.join(shps_path, 'Uso_solo_hidrografia.shp'),
]

delete_shp(list_files)

In [ ]:
shutil.rmtree(temp_path, ignore_errors=True)